In [ ]:
import networkx as nx
from parse import read_input_file, write_output_file
from utils import is_valid_solution, calculate_happiness, convert_dictionary
import sys
import random
import numpy as np
import glob
import os.path
import matplotlib.pyplot as plt

In [ ]:
path = "inputs/10.in"
G1, s1 = read_input_file(path)

In [ ]:
path2 = "inputs/20.in"
G2, s2 = read_input_file(path2)

In [ ]:
path3 = "inputs/50.in"
G3, s3 = read_input_file(path3)

In [ ]:
def plot_graph(G, attr='happiness'):
    '''
        Inputs:
            G: networkx graph
            attr: edge attributes like stress, happiness
        Output:
            None
    '''
    plt.figure(figsize=(18,18))
    pos = nx.circular_layout(G)
    labels = nx.get_edge_attributes(G, attr)
    nx.draw_networkx(G, pos)
    nx.draw_networkx_edge_labels(G,pos, edge_labels=labels)
    plt.title("{} GRAPH".format(attr.upper()))
    plt.show()

In [ ]:
def get_max_happiness(G1, node):
    ret_node = -1
    curr_max = -1
    for j in G1.nodes:
        if j == node: continue
        else:
            try:
                if G1[node][j]['happiness'] > curr_max:
                    ret_node = j
                    curr_max = G1[node][j]['happiness']
            except KeyError:
                break
    return ret_node

In [7]:
def potential_room_happiness(G, node, s_room, room):
    potential_happiness = s_room
    for j in room:
        try:
            if j == node: continue
            potential_happiness += G[node][j]['happiness']
        except KeyError:
            continue
            
    return potential_happiness

In [8]:
def solve(G, s):
    k = len(G.nodes) // 2    
    stress_budget = s / k
    
    mapping = dict(zip([i for i in range(k)], [[] for i in range(k)]))
    room_assignments = [[] for i in range(k)]
    all_stress = dict(zip([i for i in range(k)], [0 for i in range(k)]))
    curr_room = 0
    # call helper function to find room assignment for node i
    while len(G.nodes) != 2 :
        try:
            try:
                try:
                    node = list(G.nodes)[0]
                    match= get_max_happiness(G, node)
                except:
                    continue
                stress_add_node = all_stress[curr_room]
                stress_add_match = all_stress[curr_room]
                if len(room_assignments[curr_room]) == 0:
                    all_stress[curr_room] += (G[node][match]['stress'])
                else:
                    for j in room_assignments[curr_room]:
                        try:
                            stress_add_node +=  G[node][j]['stress']

                            stress_add_match += G[match][j]['stress']
                        except KeyError as e:
                            continue
                if stress_add_node > stress_budget:
                    #put both into new room node
                    room_assignments[curr_room + 1] = [node, match]
                    all_stress[curr_room + 1] += stress_add_node
                elif stress_add_node + stress_add_match < stress_budget:
                    # add both to room
                    room_assignments[curr_room] += [node, match]
                    all_stress[curr_room] += stress_add_node
                    all_stress[curr_room] += stress_add_match
                elif stress_add_node < stress_budget or stress_add_match < stress_budget:
                    # choose max(happiness)
                    # one goes to curr 
                    # other goes into next room?
                    node_happy = potential_room_happiness(G,node,all_stress[curr_room],room_assignments[curr_room])
                    match_happy = potential_room_happiness(G,match,all_stress[curr_room],room_assignments[curr_room])
                    if node_happy >= match_happy:
                        room_assignments[curr_room].append(node)
                        all_stress[curr_room] += stress_add_node
                        room_assignments[curr_room + 1].append(match)    
                        all_stress[curr_room + 1] += stress_add_match

                    else:
                        room_assignments[curr_room].append(match)
                        all_stress[curr_room] += stress_add_match
                        room_assignments[curr_room + 1].append(node)         
                        all_stress[curr_room + 1] += stress_add_node


                for k, v in all_stress.items():
                    if v > stress_budget:
                        #mapping[curr_room] = room_assignments[curr_room]
                        curr_room += 1
                G.remove_node(node)
                G.remove_node(match)
                #plot_graph(G)
                if len(G.nodes) == 2:
                    for i in range(len(room_assignments)):  
                        if len(room_assignments[i]) == 0:
                            room_assignments[i] += list(G.nodes)
                            break

                elif len(G.nodes) == 1:
                    for i in range(len(room_assignments)):

                        if len(room_assignments[i]) == 0:

                            room_assignments[i] += list(G.nodes)
                            break


                mapping = dict(zip([i for i in range(len(room_assignments))], room_assignments))
            except IndexError:
                for i in range(len(room_assignments)):
                    if len(room_assignments[i]) == 0:
                        curr_room = i
                        continue
                    else:
                        pass
        except KeyError:
            continue
    return convert_dictionary(mapping), k

In [9]:
res1, k = solve(G1, s1)
res1

{0: 0, 9: 0, 1: 0, 3: 1, 2: 1, 6: 2, 4: 3, 5: 3, 7: 4, 8: 4}

In [10]:
is_valid_solution(res1, G1, s1, k)

True

In [11]:
output_path = './out' + os.path.basename(os.path.normpath('inputs/10.in'))[:-3] + '.out'
write_output_file(res1, output_path)

In [ ]:
res2, k2 = solve(G2, s2)
res2

In [ ]:
is_valid_solution(res2, G2, s2, k2)

In [ ]:
output_path = './out' + os.path.basename(os.path.normpath('inputs/20.in'))[:-3] + '.out'
write_output_file(res2, output_path)

In [ ]:
res3, k3 = solve(G3, s3)
res3

In [ ]:
is_valid_solution(res3, G3, s3, k3)

In [ ]:
import os
def generate(filename):
    G, s = read_input_file('inputs/' + filename)
    D, k = solve(G, s)
    assert is_valid_solution(D, G, s, k)
    print("Total Happiness: {}".format(calculate_happiness(D, G)))
    output_path = './out' + os.path.basename(os.path.normpath(filename))[:-3] + '.out'
    write_output_file(D, output_path)
        
if __name__ == '__main__':
    #print(os.listdir('./inputs'))
    for filename in os.listdir('./inputs'):
        ## ideally run all files
        if "small" in filename:
             generate(filename)
    